In [1]:
import miditoolkit
import remi_utils as utils
from collections import Counter
import pickle
import glob
import json
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
# create pickle file based on dataset

def extract_events(input_path, chord=False):
    note_items, tempo_items = utils.read_items(input_path)
    note_items = utils.quantize_items(note_items)
    max_time = note_items[-1].end
    if chord:
        chord_items = utils.extract_chords(note_items)
        items = chord_items + tempo_items + note_items
    else:
        items = tempo_items + note_items
    groups = utils.group_items(items, max_time)
    events = utils.item2event(groups)
    return events

In [3]:
all_elements= []
for midi_file in glob.glob("./data_augmented/melody/*/*.mid*", recursive=True):
    #print(midi_file)
    events = extract_events(midi_file) # If you're analyzing chords, use `extract_events(midi_file, chord=True)`
    for event in events:
        element = '{}_{}'.format(event.name, event.value)
        all_elements.append(element)

for midi_file in glob.glob("./data_augmented/piano/*/*.mid*", recursive=True):
    #print(midi_file)
    try:
        events = extract_events(midi_file) # If you're analyzing chords, use `extract_events(midi_file, chord=True)`
    except:
        print(midi_file)
    for event in events:
        element = '{}_{}'.format(event.name, event.value)
        all_elements.append(element)        

counts = Counter(all_elements)
event2word = {c: i for i, c in enumerate(counts.keys())}
word2event = {i: c for i, c in enumerate(counts.keys())}
pickle.dump((event2word, word2event), open('dictionary_augmented.pkl', 'wb'))

KeyboardInterrupt: 

In [3]:
event2word, word2event = pickle.load(open('dictionary_augmented.pkl', 'rb'))

In [9]:
count = 0
intros = []
outros = []
solos = []
with open('solos.json') as json_file: 
    data = json.load(json_file) 
semitones = [-3,-2,-1,0,1,2,3]
for i in range(1,910):
    for k in semitones:
        filename = str(i).zfill(3)
        if filename not in data:
            continue
        for j in range(len(data[filename])):
            count += 1
            # extract intro
            intro = extract_events("./data_augmented/melody/intro/" + str(i).zfill(3) + "_solo_" + str(k) + "_" + str(j) + ".mid")
            w_intro = utils.event_to_word(intro, event2word)
            intros.append(w_intro)
            # extract outro
            outro = extract_events("./data_augmented/melody/outro/" + str(i).zfill(3) + "_solo_" + str(k) + "_" + str(j) + ".mid")
            w_outro = utils.event_to_word(outro, event2word)
            outros.append(w_outro)
            # extract solo
            solo = extract_events("./data_augmented/melody/middle/" + str(i).zfill(3) + "_solo_" + str(k) + "_" + str(j) + ".mid")
            w_solo = utils.event_to_word(solo, event2word)
            solos.append(w_solo)
        

In [10]:
count = 0
intros_piano = []
outros_piano = []
solos_piano = []
with open('solos.json') as json_file: 
    data = json.load(json_file) 
for i in range(1,910):
    for k in semitones:
        filename = str(i).zfill(3)
        if filename not in data:
            continue
        for j in range(len(data[filename])):
            count += 1
            # extract intro
            intro = extract_events("./data_augmented/piano/intro/" + str(i).zfill(3) + "_solo_" + str(k) + "_" + str(j) + ".mid")
            w_intro = utils.event_to_word(intro, event2word)
            intros_piano.append(w_intro)
            # extract outro
            outro = extract_events("./data_augmented/piano/outro/" + str(i).zfill(3) + "_solo_" + str(k) + "_" + str(j) + ".mid")
            w_outro = utils.event_to_word(outro, event2word)
            outros_piano.append(w_outro)
            # extract solo
            solo = extract_events("./data_augmented/piano/middle/" + str(i).zfill(3) + "_solo_" + str(k) + "_" + str(j) + ".mid")
            w_solo = utils.event_to_word(solo, event2word)
            solos_piano.append(w_solo)


In [11]:
data = [intros, intros_piano, outros, outros_piano, solos, solos_piano]
pickle.dump(data, open('./solo_classification_REMI_augmented/solo_generation_dataset.pkl', 'wb'))

In [22]:
data = pickle.load(open('./solo_classification_REMI_augmented/solo_generation_dataset.pkl', 'rb'))

In [19]:
def find_max_length(series):
    max_length=0
    for i in range(len(series)):
        if max_length < len(series[i]):
            max_length = len(series[i])
    return max_length

def pad_dataset(dataset, word2event):
    pad_value = len(word2event)
    max_length = 0
    for i in range(len(dataset)):
        if max_length < find_max_length(dataset[i]):
            max_length = find_max_length(dataset[i])
    print(max_length)
    for i in range(len(dataset)):
        for j in range(len(dataset[i])):
            while len(dataset[i][j]) < max_length:
                dataset[i][j].append(pad_value)
    return dataset

In [20]:
data_padded = pad_dataset(data,word2event)

1637


In [21]:
pickle.dump(data_padded, open('./solo_generation_dataset/solo_generation_dataset_padded.pkl', 'wb'))
data_padded = pickle.load(open('./solo_generation_dataset/solo_generation_dataset_padded.pkl', 'rb'))
data = data_padded

In [23]:
data_text = data
for i in range(len(data)):
    for j in range(len(data[i])):
        string_array = [str(num) for num in data[i][j]]
        data_text[i][j] = ' '.join(string_array)

In [24]:
intros_t, intros_piano_t, outros_t, outros_piano_t, solos_t, solos_piano_t = data_text

In [25]:
len(intros_t)

5250

In [26]:
intros_train, intros_t_subset, intros_piano_train, intros_piano_t_subset, outros_train, outros_t_subset, outros_piano_train, outros_piano_t_subset, solos_train, solos_t_subset, solos_piano_train, solos_piano_t_subset = intros_t[:5250-1568], intros_t[-1568:], intros_piano_t[:5250-1568], intros_piano_t[-1568:], outros_t[:5250-1568], outros_t[-1568:], outros_piano_t[:5250-1568], outros_piano_t[-1568:], solos_t[:5250-1568], solos_t[-1568:], solos_piano_t[:5250-1568], solos_piano_t[-1568:]

In [27]:
half = len(intros_t_subset)//2
intros_test, intros_valid, intros_piano_test, intros_piano_valid, outros_test, outros_valid, outros_piano_test, outros_piano_valid, solos_test, solos_valid, solos_piano_test, solos_piano_valid = intros_t_subset[half:], intros_t_subset[:half], intros_piano_t_subset[half:], intros_piano_t_subset[:half], outros_t_subset[half:],outros_t_subset[:half], outros_piano_t_subset[half:], outros_piano_t_subset[:half], solos_t_subset[half:], solos_t_subset[:half], solos_piano_t_subset[half:],solos_piano_t_subset[:half]

In [28]:
intros_test = intros_test[3::7]
intros_valid = intros_valid[3::7]
intros_piano_test = intros_piano_test[3::7]
intros_piano_valid = intros_piano_valid[3::7]
outros_test = outros_test[3::7]
outros_valid = outros_valid[3::7]
outros_piano_test = outros_piano_test[3::7]
outros_piano_valid = outros_piano_valid[3::7]
solos_test = solos_test[3::7]
solos_valid = solos_valid[3::7]
solos_piano_test = solos_piano_test[3::7]
solos_piano_valid = solos_piano_valid[3::7]

In [29]:
train = []
valid = []
test = []

for i in range(len(intros_train)):
    train.append([0,intros_train[i]])
    train.append([0,outros_train[i]])
    train.append([1,solos_train[i]])
    
for i in range(len(intros_valid)):
    valid.append([0,intros_valid[i]])
    valid.append([0,outros_valid[i]])
    valid.append([1,solos_valid[i]])

    
for i in range(len(intros_test)):
    test.append([0,intros_test[i]])
    test.append([0,outros_test[i]])
    test.append([1,solos_test[i]])


In [19]:
train = []
valid = []
test = []

vocal_train = []
vocal_valid = []
vocal_test = []

for i in range(len(intros_train)):
    vocal_train.append([0,intros_train[i]])
    vocal_train.append([0,outros_train[i]])
    train.append([1,solos_train[i]])

vocal_train, _ = train_test_split(vocal_train, test_size=int(len(vocal_train)/2))

for i in range(len(vocal_train)):
    train.append(vocal_train[i])
    
    
for i in range(len(intros_valid)):
    vocal_valid.append([0,intros_valid[i]])
    vocal_valid.append([0,outros_valid[i]])
    valid.append([1,solos_valid[i]])

vocal_valid, _ = train_test_split(vocal_valid, test_size=int(len(vocal_valid)/2))

for i in range(len(vocal_valid)):
    valid.append(vocal_valid[i])
    
    
for i in range(len(intros_test)):
    vocal_test.append([0,intros_test[i]])
    vocal_test.append([0,outros_test[i]])
    test.append([1,solos_test[i]])

vocal_test, _ = train_test_split(vocal_test, test_size=int(len(vocal_test)/2))

for i in range(len(vocal_test)):
    test.append(vocal_test[i])
    

In [30]:
df_train = pd.DataFrame(train,columns=['label','melody'])
df_val = pd.DataFrame(test,columns=['label','melody'])
df_test = pd.DataFrame(valid,columns=['label','melody'])

In [31]:
destination_folder="solo_classification_REMI_augmented_unbalanced"
df_train.to_csv(destination_folder + '/train.csv', index=False)
df_val.to_csv(destination_folder + '/val.csv', index=False)
df_test.to_csv(destination_folder + '/test.csv', index=False)

In [218]:
def remove_padding(series, word2event):
    return [value for value in series if value != len(word2event)]

In [211]:
lst_int = [int(x) for x in df_train.values[0][0].split(' ')]
utils.write_midi(remove_padding(lst_int, word2event), word2event, 'test.midi')